# 📘 Preparar Dataset para Fine-Tuning (versão completa com gdown)
Este notebook instala `gdown`, baixa o `trn.json` do Google Drive, e gera `dataset_preparado.jsonl`.

In [ ]:
import json
import subprocess
import sys

# Tenta instalar gdown automaticamente caso não esteja disponível
try:
    import gdown
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gdown"])
    import gdown

# ID do arquivo trn.json compartilhado no Google Drive
file_id = "1kq9bSDUfj0sxGO6C9bb95EYWtey6Smf_"
output = "trn.json"

# Baixa o arquivo trn.json do Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

# Caminho do arquivo de entrada (baixado) e saída
CAMINHO_JSON = "trn.json"
SAIDA_JSONL = "dataset_preparado.jsonl"

# Função responsável por preparar os dados no formato necessário para treinamento do modelo
def preparar_dataset_jsonl(caminho_entrada, caminho_saida):
    contador = 0
    ignorados = 0

    with open(caminho_entrada, 'r', encoding='utf-8') as f_in, \
         open(caminho_saida, 'w', encoding='utf-8') as f_out:

        for linha in f_in:
            try:
                item = json.loads(linha)
                titulo = item.get("title", "").strip()
                descricao = item.get("content", "").strip()

                if titulo and descricao:
                    prompt = f"O que é {titulo}?"
                    completion = descricao
                    f_out.write(json.dumps({
                        "prompt": prompt,
                        "completion": completion
                    }, ensure_ascii=False) + '\n')
                    contador += 1
                else:
                    ignorados += 1

            except json.JSONDecodeError:
                ignorados += 1
                continue

    print(f"✅ Dataset gerado com sucesso!")
    print(f"📄 Linhas válidas: {contador}")
    print(f"🚫 Linhas ignoradas (sem conteúdo ou erro): {ignorados}")
    print(f"💾 Arquivo salvo em: {caminho_saida}")

# Executa a função
preparar_dataset_jsonl(CAMINHO_JSON, SAIDA_JSONL)
